In [1]:
from flask import Flask, send_file, request, jsonify, make_response, current_app
from flask_ngrok import run_with_ngrok
from flask_cors import CORS
from functools import update_wrapper, wraps
import requests

import tensorflow as tf
from tensorflow.keras.preprocessing.image import smart_resize
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.models import load_model
import base64

In [ ]:
target_size = (224, 224)

targetnames = np.array(['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc'])

model_path = 'densenetSoftAtt.h5'

model = load_model(model_path)

In [ ]:
def proprocessed_flow(img_txt, _target_size):
    # decode
    img = base64.b64decode(img_txt)
    # resize
    img = smart_resize(img, _target_size)
    # preprocess
    img = preprocess_input(img)
    # return
    return img
def get_result(_json_data, _target_size, _model):
    # Get img
    img = proprocessed_flow(_json_data['image'])
    # Predict
    predictions = _model.predict({'image' : img,
                                 'age' : _json_data['age'],
                                  'sex' : _json_data['sex'],
                                  'local' : _json_data['localization']})
    # Compress result
    indices = np.argpartition(predictions, -3)[-3:]
    top_prob = predictions[indices]
    top_name = targetnames[indices]
    return jsonify({_name : prob for _name, prob in zip(top_name, top_prob)})
    

In [ ]:
app = Flask(__name__)
run_with_ngrok(app)

@app.route("/")
def home():
    return "<h1>TechStart Competition</h1>"

@app.route('/get_predict', methods= ['GET', 'POST'])
def get_predict():
    if request.method == 'POST':
        json_data = request.json
        return get_result(json_data, target_size, model)
    
# APP running
app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://122d-14-232-202-93.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [27/Mar/2022 14:55:33] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2022 14:55:33] "GET /favicon.ico HTTP/1.1" 404 -
